In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip setuptools wheel
# !{sys.executable} -m pip install networkx numpy matplotlib pandas pydot

In [2]:
import networkx as nx
import itertools as it
import matplotlib.pyplot as plt
import math
import scipy
import pickle
import os

In [3]:
DPI = 300

In [4]:
BaseDirectory = os.getcwd()

In [5]:
# print(BaseDirectory)

In [6]:
def _draw_graph(Graph):
    Dimension = 1
    FigDimension = 5
    fig,axes = plt.subplots(nrows=Dimension,ncols=Dimension,figsize=(FigDimension,FigDimension),dpi=200)
    nx.draw_circular(Graph, ax=axes, with_labels=True, edge_color='black', node_color='black', font_color='white')
    axes.set_title("Graph 0")

In [7]:
def _draw_graph_list(GraphList, Names = None):
    if Names == None:
        Names = []
        for i in range(len(GraphList)):
            Names.append(f"Graph {i+1}")
    if len(GraphList) <= 0:
        return
    elif len(GraphList) == 1:
        Dimension = 1
        FigDimension = min(Dimension*Dimension, 65000/Dimension)
        fig,axes = plt.subplots(nrows=Dimension,ncols=Dimension,figsize=(FigDimension,FigDimension),dpi=200)
        nx.draw_circular(GraphList[0], ax=axes, with_labels=True, edge_color='black', node_color='black', font_color='silver')
        axes.set_title("Graph 0")
    else:
        Dimension = math.ceil(math.sqrt(len(GraphList)))
        FigDimension = min(Dimension*Dimension, 65000/Dimension)
        fig,axes = plt.subplots(nrows=Dimension,ncols=Dimension,figsize=(FigDimension,FigDimension),dpi=200)
        GraphCounter=0
        for x in range(0,Dimension):
            for y in range(0,Dimension):
                if GraphCounter < len(GraphList):
                    axes[x,y].set_title(Names[GraphCounter])
                    nx.draw_circular(GraphList[GraphCounter], ax=axes[x,y], with_labels=True, edge_color='black', node_color='black', font_color='silver')
                    GraphCounter+=1
                else:
                    nx.draw(nx.null_graph(), ax=axes[x,y])
    plt.tight_layout()
    plt.show()
    plt.close()
    return

In [8]:
def _save_graph_list(GraphList, FileName = "Default", Names = None):
    if Names == None:
        Names = []
        for i in range(len(GraphList)):
            Names.append(f"Graph {i+1}")
    if len(GraphList) <= 0:
        return
    elif len(GraphList) == 1:
        Dimension = 1
        FigDimension = min(Dimension*Dimension, 65000/Dimension)
        fig,axes = plt.subplots(nrows=Dimension,ncols=Dimension,figsize=(FigDimension,FigDimension),dpi=200)
        nx.draw_circular(GraphList[0], ax=axes, with_labels=True, edge_color='black', node_color='black', font_color='white')
        axes.set_title("Graph 0")
    else:
        Dimension = math.ceil(math.sqrt(len(GraphList)))
        FigDimension = min(Dimension*Dimension, 65000/Dimension)
        fig,axes = plt.subplots(nrows=Dimension,ncols=Dimension,figsize=(FigDimension,FigDimension),dpi=200)
        GraphCounter=0
        for x in range(0,Dimension):
            for y in range(0,Dimension):
                if GraphCounter < len(GraphList):
                    axes[x,y].set_title(Names[GraphCounter])
                    nx.draw_circular(GraphList[GraphCounter], ax=axes[x,y], label=f"Graph {GraphCounter}", with_labels=True, edge_color='black', node_color='black', font_color='white')
                    GraphCounter+=1
                else:
                    nx.draw(nx.null_graph(), ax=axes[x,y])
    plt.tight_layout()
    plt.savefig(f"{FileName}.svg")
    plt.close()
    return

In [9]:
def _get_largest_subgraph_of_both(Graph1, Graph2):
#     
    
    if Graph1.size() > Graph2.size():
        LargestGraph = Graph1
        SmallestGraph = Graph2
    else:
        LargestGraph = Graph2
        SmallestGraph = Graph1
    
    if SmallestGraph.number_of_nodes() < 1:
        return nx.null_graph()
    
    LargestSubgraph = nx.path_graph(1)
    for SubgraphSize in range(SmallestGraph.size(), 0, -1):
        for Edges in it.combinations(SmallestGraph.edges(), SubgraphSize):
            HostLineGraph = nx.line_graph(LargestGraph)
            
            if nx.algorithms.isomorphism.GraphMatcher(LargestGraph, nx.edge_subgraph(SmallestGraph, Edges).copy()).subgraph_is_monomorphic():
                LargestSubgraph = nx.edge_subgraph(SmallestGraph, Edges).copy()
                break
        if LargestSubgraph.size() > 0:
            break
            
    return LargestSubgraph

In [10]:
# Graph1 = nx.cycle_graph(3)
# Graph1.add_node(3)
# Graph1.add_node(4)
# Graph1.add_edge(3,4)

# Graph2 = nx.complete_multipartite_graph(2,3)

# matcher = nx.algorithms.isomorphism.GraphMatcher(nx.line_graph(Graph2), nx.line_graph(Graph1))
# # print(next(matcher.subgraph_isomorphisms_iter()))

# Graph3 = _get_largest_subgraph_of_both(Graph1, Graph2)

# GraphList = [Graph1, Graph2, Graph3]
# _draw_graph_list(GraphList, ["Graph 1", "Graph 2", "Largest Intersection"])

# GraphList = _get_unique_edge_induced_subgraphs(Graph2)
# _draw_graph_list(GraphList)

In [11]:
def _save_coloring(ColoringList, Intersection = None, HostGraph = None, FileName = "Default"):
#     This function saves specifically a red blue coloring
    
    fig,axes = plt.subplots(nrows=1, ncols=4, figsize=(10,2.5))
    nx.draw_circular(ColoringList[0], ax=axes[0], with_labels=True, edge_color="red", node_color='black', font_color='white')
    axes[0].set_title("Red Subgraph")
    
    nx.draw_circular(ColoringList[1], ax=axes[1], with_labels=True, edge_color="blue", node_color='black', font_color='white')
    axes[1].set_title("Blue Subgraph")
    
    if Intersection == None:
        Intersection = _get_largest_subgraph_of_both(ColoringList[0], ColoringList[1])
    nx.draw_circular(Intersection, ax=axes[2], with_labels=True, edge_color="violet", node_color='black', font_color='white')
    axes[2].set_title("Intersection")
    
    if HostGraph == None:
        HostGraph = nx.compose(ColoringList[0], ColoringList[1])
    nx.draw_circular(HostGraph, ax=axes[3], with_labels=True, edge_color="black", node_color='black', font_color='white')
    axes[3].set_title("Host")
    
    plt.tight_layout()
    plt.savefig(f"{FileName}.svg")
    plt.close()
    return

In [12]:
# # This should work, and should be short circuit but something is off right now
# def _get_all_edge_induced_subgraphs(Host):
#     Subgraphs = []
#     for NumEdgesChosen in range(Host.size(), math.floor(Host.size()/2), -1):
#         print(f"Choosing {NumEdgesChosen} edges")
#         for Edges in it.combinations(Host.edges(),NumEdgesChosen+1):
#             Subgraphs.append(nx.edge_subgraph(Host, Edges).copy())
#             Subgraphs.append(nx.edge_subgraph(Host, set(Host.edges)-set(Edges)).copy())
#     return Subgraphs

In [13]:
def _get_all_edge_induced_subgraphs(Host):
#     This function takes in a graph, and returns a list of all subgraphs induced by selecting all combinations of edges from the edge set of the host graph
    
#     Initialize the list of all subgraphs to an empty list
    Subgraphs = []
    
#     Iterate through selecting 0, 1, 2, ... edges that will be selected from the edge set of the host graph
    for NumEdgesChosen in range(Host.size()+1):
        
#         For each given number of edges that will be selected, select each combination of that many edges from the host graph
        for Edges in it.combinations(Host.edges(),NumEdgesChosen):
        
#             Add the respective edge-induced subgraph to the list of all subgraphs
            Subgraphs.append(nx.edge_subgraph(Host, Edges).copy())
    
    return Subgraphs

In [14]:
def _return_unique_graphs(GraphList):
#     This function takes in a list of graphs, and returns a list of unique graphs from the input list, taking graph isomorphism as the equivalence relation
    
#     Initialize the list of unique graphs to an empty list
    UniqueGraphs = []
    
#     Iterate over the entire list of graphs
    for Graph in GraphList:
        
#         First assume that the graph is unique up to subgraph isomorphism
        Unique = True
    
#         Iterate through each graph already in the list of unique subgraphs, and if there is a graph in the list of unique subgraphs that is isomorphic to the potentially unique subgraph, we note that and break out of the loop
        for UniqueGraph in UniqueGraphs:
            if nx.is_isomorphic(Graph, UniqueGraph):
                Unique = False
                break
                
#         If none of the already determined unique subgraphs are isomorphic to the held subgraph, then we can safely determine that it is unique up to isomorphism
        if Unique:
            UniqueGraphs.append(Graph)
            
    return UniqueGraphs

In [15]:
def _get_unique_edge_induced_subgraphs(Host):
#     This funciton is essentially a wrapper for returning only the unique subgraphs of an input graph
    return _return_unique_graphs(_get_all_edge_induced_subgraphs(Host))

In [16]:
def _get_all_colorings(Host, SubgraphList = None):
#     This function takes in a host graph, and a list of subgraphs (or calculates all of them), and returns a list of all red blue colorings associated with coloring every subgraph in the list red and the complement (in the host) blue
#     This function returns a list of tuples where the first index is the red subgraph, and the second index is the blue subgraph
    
#     Initialize the list of colorings to an empty list
    Colorings = []
    
#     If the list of subgraphs isn't passed into this funciton, this funciton will calculate the list of all non-isomorphic subgraphs
    if SubgraphList == None:
        SubgraphList = _get_unique_edge_induced_subgraphs(Host)
    
#     Make the induced colorings and put them into the list of colorings
    for RedColoring in SubgraphList:
        BlueColoring = nx.edge_subgraph(Host, set(Host.edges())-set(RedColoring.edges())).copy()
        Colorings.append(tuple((RedColoring, BlueColoring)))
        
    return Colorings

In [17]:
def _get_graph_from_file_name(GraphName):
    if "K_" in GraphName:
        if "," in GraphName:
            n,m=FileName.split(".",1)[0].split("K_",1)[1].split(",")
            return nx.complete_multipartite_graph(n,m)
        else:
            n=FileName.split(".",1)[0].split("K_",1)[1]
            n=int(n)
            return nx.complete_graph(n)
    elif "C_" in GraphName:
            n=FileName.split(".",1)[0].split("C_",1)[1]
            n=int(n)
            return nx.cycle_graph(n)
    elif "P_" in GraphName:
            n=FileName.split(".",1)[0].split("P_",1)[1]
            n=int(n)
            return nx.path_graph(n)

In [18]:
# Build the unique subgraphs of complete bipartites

MaxN = 3
MaxM = 3

for n in range(1,MaxN+1):
    for m in range(n,MaxM+1):
        GraphName = f"K_{n},{m}"
        
        if os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
            continue
            
        print(f"Inspecting: {GraphName}")
        Host = nx.complete_multipartite_graph(n,m)
        
        InducedGraphs = _get_all_edge_induced_subgraphs(Host)
        print(f"     There are {len(InducedGraphs)} distinct subgraphs with at least 1 edge of the chosen graph.")
        
        UniqueGraphs = _return_unique_graphs(InducedGraphs)
        print(f"     There are {len(UniqueGraphs)} distinct subgraphs up to isomorphism with at least 1 edge of the chosen graph.")
        
        with open(f"{GraphName}.UniqueGraphs.pickle", "wb") as OutFile:
            pickle.dump(UniqueGraphs, OutFile)

In [19]:
# Build the unique subgraphs of complete graphs

MaxK = 5

for k in range(1,MaxK+1):
        GraphName = f"K_{k}"
        
        if os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
            continue
            
        print(f"Inspecting: {GraphName}")
        Host = nx.complete_graph(k)
        
        InducedGraphs = _get_all_edge_induced_subgraphs(Host)
        print(f"     There are {len(InducedGraphs)} distinct subgraphs with at least 1 edge of {GraphName}.")
        
        UniqueGraphs = _return_unique_graphs(InducedGraphs)
        print(f"     There are {len(UniqueGraphs)} distinct subgraphs up to isomorphism with at least 1 edge of {GraphName}.")
        
        with open(f"{GraphName}.UniqueGraphs.pickle", "wb") as OutFile:
            pickle.dump(UniqueGraphs, OutFile)

In [20]:
# Build the unique subgraphs of cycle graphs

MaxN = 9

for n in range(1,MaxN+1):
        GraphName = f"C_{n}"
        
        if os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
            continue
            
        print(f"Inspecting: {GraphName}")
        Host = nx.cycle_graph(n)
        
        InducedGraphs = _get_all_edge_induced_subgraphs(Host)
        print(f"     There are {len(InducedGraphs)} distinct subgraphs with at least 1 edge of {GraphName}.")
        
        UniqueGraphs = _return_unique_graphs(InducedGraphs)
        print(f"     There are {len(UniqueGraphs)} distinct subgraphs up to isomorphism with at least 1 edge of {GraphName}.")
        
        with open(f"{GraphName}.UniqueGraphs.pickle", "wb") as OutFile:
            pickle.dump(UniqueGraphs, OutFile)

In [21]:
# Build the unique subgraphs of cycle graphs

MaxN = 9

for n in range(1,MaxN+1):
        GraphName = f"P_{n}"
        
        if os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
            continue
            
        print(f"Inspecting: {GraphName}")
        Host = nx.path_graph(n)
        
        InducedGraphs = _get_all_edge_induced_subgraphs(Host)
        print(f"     There are {len(InducedGraphs)} distinct subgraphs with at least 1 edge of {GraphName}.")
        
        UniqueGraphs = _return_unique_graphs(InducedGraphs)
        print(f"     There are {len(UniqueGraphs)} distinct subgraphs up to isomorphism with at least 1 edge of {GraphName}.")
        
        with open(f"{GraphName}.UniqueGraphs.pickle", "wb") as OutFile:
            pickle.dump(UniqueGraphs, OutFile)

In [22]:
# Build the colorings, and  intersections

for FileName in os.listdir(os.getcwd()):
    if ".UniqueGraphs.pickle" in FileName:
        HostGraph = _get_graph_from_file_name(FileName)
        GraphName = FileName.split(".",1)[0]
        print(f"Inspecting {GraphName}")
        
#         Pick up the list of unique subgraphs
        if not os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
            continue
        UniqueSubgraphs = []
        CommonIntersectionSubgraphs = []
        with open(f"{GraphName}.UniqueGraphs.pickle", "rb") as InputFile:
            UniqueSubgraphs = pickle.load(InputFile)
            
#         Build a directory (if need be) and move into it for the given K_n,m
        if not os.path.exists(f"{GraphName}"):
            os.mkdir(f"{GraphName}")
        os.chdir(BaseDirectory+'/'+GraphName)
        
#         Determine all colorings, and save images of the colorings (and their intersection)
        Colorings = _get_all_colorings(HostGraph)
        ColoringIntersections = []
        ColoringID = 0
        for Coloring in Colorings:
            ColoringIntersections.append(_get_largest_subgraph_of_both(Coloring[0], Coloring[1]))
            _save_coloring(Coloring, ColoringIntersections[ColoringID], HostGraph, f"{GraphName}.Coloring.{ColoringID}.svg")
            ColoringID += 1
            
        with open(f"{GraphName}.Colorings.pickle", "wb") as OutFile:
            pickle.dump(Colorings, OutFile)
            
        with open(f"{GraphName}.Unique.Coloring.Intersections.pickle", "wb") as OutFile:
            pickle.dump(_return_unique_graphs(ColoringIntersections), OutFile)
        
        os.chdir(BaseDirectory)

Inspecting C_9
Inspecting C_8
Inspecting P_9
Inspecting P_8
Inspecting P_7
Inspecting C_2
Inspecting P_6
Inspecting C_3
Inspecting K_1
Inspecting K_2
Inspecting P_5
Inspecting C_1
Inspecting K_3
Inspecting P_4
Inspecting K_5
Inspecting C_7
Inspecting K_2,3
Inspecting P_2
Inspecting C_6
Inspecting K_2,2
Inspecting K_4
Inspecting P_3
Inspecting C_5
Inspecting P_1
Inspecting C_4
Inspecting K_1,1
Inspecting K_1,2
Inspecting K_1,3
Inspecting K_3,3


In [23]:
# # # Parse unique subgraphs into red/blue colorings and draw pictures with the red, blue, and host graph
# # # Also determine the largest subgraph that is in both the red and blue subgrahs of each coloring and determine all of the possible ones, up to isomorphism
# # # Determine total ordering and reduced poset graph for the ordering given by subset containment
# for FileName in os.listdir(BaseDirectory):
#     if ".UniqueGraphs.pickle" in FileName:
# #         Get the n and m for the K_n,m host graph
#         n,m=FileName.split(".",1)[0].split("K_",1)[1].split(",")
#         n=int(n)
#         m=int(m)
#         HostGraph = nx.complete_multipartite_graph(n,m)
        
# #         Pick up the list of unique subgraphs
#         GraphName = f"K_{n},{m}"
#         print(f"Inspecting {GraphName}")
#         if not os.path.exists(f"{GraphName}.UniqueGraphs.pickle"):
#             continue
#         UniqueSubgraphs = []
#         CommonIntersectionSubgraphs = []
#         with open(f"{GraphName}.UniqueGraphs.pickle", "rb") as InputFile:
#             UniqueSubgraphs = pickle.load(InputFile)
            
# #         Build a directory (if need be) and move into it for the given K_n,m
#         if not os.path.exists(f"{GraphName}"):
#             os.mkdir(f"{GraphName}")
#         os.chdir(BaseDirectory+'/'+GraphName)
        
# #         Work on the red-blue colorings
#         counter = 1
#         for RedSubgraph in UniqueSubgraphs:
# #             Make BlueSubgraph
#             BlueSubgraph = nx.edge_subgraph(HostGraph, set(HostGraph.edges())-set(RedSubgraph.edges())).copy()
            
# #             Do the setup to hide 3 graphs in one picture (red, blue, biggest edge-induced subgraph of the red and blue subgraphs, and the host graph)
#             fig,axes = plt.subplots(nrows=1,ncols=4, figsize=(10,2.5))
#             ax = axes.flatten()
            
# #             Draw the red part
#             nx.draw_kamada_kawai(RedSubgraph, with_labels=True, ax=ax[0], edge_color='red', node_color='black', font_color='white')
#             ax[0].set_title("Red Subgraph")
        
# #             Draw the blue part
#             nx.draw_kamada_kawai(BlueSubgraph, with_labels=True, ax=ax[1], edge_color='blue', node_color='black', font_color='white')
#             ax[1].set_title("Blue Subgraph")

# #             find the smaller of the two subgraphs (the red and the blue part)
#             if RedSubgraph.size() > BlueSubgraph.size():
#                 SmallestSubgraph = RedSubgraph.copy()
#                 LargestSubgraph = BlueSubgraph.copy()
#             else:
#                 SmallestSubgraph = BlueSubgraph.copy()
#                 LargestSubgraph = RedSubgraph.copy()
            
# #             Determine the largest subgraph of both the red and the blue subgraphs
#             for SubgraphSize in range(SmallestSubgraph.size(), 0, -1):
#                 LargestCommonSubgraph = None
#                 if SubgraphSize == 0:
#                     LargestCommonSubgraph = nx.null_graph()
#                     break
#                 for Edges in it.combinations(SmallestSubgraph.edges(), SubgraphSize):
#                     SubSubgraph = nx.edge_subgraph(SmallestSubgraph, Edges).copy()
#                     if nx.algorithms.isomorphism.GraphMatcher(nx.line_graph(LargestSubgraph), nx.line_graph(SubSubgraph)).subgraph_is_isomorphic():
#                         LargestCommonSubgraph = SubSubgraph.copy()
#                         CommonIntersectionSubgraphs.append(LargestCommonSubgraph.copy())
#                         break
#                 if LargestCommonSubgraph is not None:
#                     break
                    
# #             Draw the largest common subgraph of the red and the blue subgraphs
#             if LargestCommonSubgraph is not None:
#                 nx.draw_circular(LargestCommonSubgraph, with_labels=True, ax=ax[2], edge_color='green', node_color='green', font_color='white')
#                 ax[2].set_title("Intersection")
#             else:
#                 nx.draw(nx.null_graph(), ax=ax[2])
    
# #             Draw the host graph
#             nx.draw_kamada_kawai(HostGraph, with_labels=True, ax=ax[3], edge_color='black', node_color='black', font_color='white')
#             ax[3].set_title(GraphName)

# #             Save the figure to file
#             plt.savefig(f"{GraphName}.Red-Blue.Coloring.{counter}.svg")
#             plt.clf()
#             plt.close()
#             counter+=1
            
# #         Determine the total ordering
#         FixedSubgraphID = 1
#         TotalOrderingAdjacencyList = ""
#         AllGraphSizes = ""
        
# #         Fix a particular unique subgraph and see which other unique subgraphs contain it
#         for FixedSubgraph in UniqueSubgraphs:
#             AllGraphSizes += f"Graph {FixedSubgraphID} has {FixedSubgraph.size()} edges\n"
#             TotalOrderingAdjacencyList += f"{FixedSubgraphID}"
#             HostGraphID = 1
#             for HostGraph in UniqueSubgraphs:
#                 if nx.algorithms.isomorphism.GraphMatcher(nx.line_graph(HostGraph), nx.line_graph(FixedSubgraph)).subgraph_is_isomorphic():
#                     TotalOrderingAdjacencyList += f" {HostGraphID}"
#                 HostGraphID += 1
#             TotalOrderingAdjacencyList += "\n"
#             FixedSubgraphID+=1
#         with open(f"{GraphName}.Unique.Subgraphs.Information.txt", "w+") as OutFile:
#             OutFile.write(AllGraphSizes)
        
# #         Build a graph represening the total ordering of this K_n,m
#         TotalOrderingGraph = nx.parse_adjlist(TotalOrderingAdjacencyList.splitlines(), create_using=nx.DiGraph())
#         TotalOrderingGraph.remove_edges_from(nx.selfloop_edges(TotalOrderingGraph))
        
# #         Draw the total ordering
#         fig,axes = plt.subplots(nrows=1,ncols=1,figsize=(5,5),dpi=DPI)
#         ax = axes
#         nx.draw_circular(TotalOrderingGraph, with_labels=True, ax=ax, edge_color='teal', node_color='black', font_color='white')
#         plt.savefig(f"{GraphName}.TotalOrdering.svg")
#         plt.clf()
#         plt.close()
        
# #         now restrict to just the relavent poset ordering
#         GoodEdges = []
#         for Edge in TotalOrderingGraph.edges():
# #             print(Edge)
#             if ((UniqueSubgraphs[int(Edge[1])-1].size())-(UniqueSubgraphs[int(Edge[0])-1].size()))==1:
# #                 print(f"Case 1: Keeping the edge between graph {Edge[0]} (size {UniqueSubgraphs[int(Edge[0])-1].size()}) and graph {Edge[1]} (size {UniqueSubgraphs[int(Edge[1])-1].size()}).")
#                 GoodEdges.append((Edge[0],Edge[1]))
#             elif ((UniqueSubgraphs[int(Edge[0])-1].size())-(UniqueSubgraphs[int(Edge[1])-1].size()))==1:
# #                 print(f"Case 2: Keeping the edge between graph {Edge[0]} (size {UniqueSubgraphs[int(Edge[0])-1].size()}) and graph {Edge[1]} (size {UniqueSubgraphs[int(Edge[1])-1].size()}).")
#                 GoodEdges.append((Edge[0],Edge[1]))
# #             else:
# #                 print(f"Case 3: Rejecting the edge between graph {Edge[0]} (size {UniqueSubgraphs[int(Edge[0])-1].size()}) and graph {Edge[1]} (size {UniqueSubgraphs[int(Edge[1])-1].size()}).")
#         ReducedPosetGraph = TotalOrderingGraph.edge_subgraph(GoodEdges).copy()
        
# #         Draw the poset heirarchy
#         fig,axes = plt.subplots(nrows=1,ncols=1,figsize=(11,11),dpi=DPI)
#         ax = axes
# #         pos = hierarchy_pos(ReducedPosetGraph)
#         positions = nx.spring_layout(ReducedPosetGraph, k=1)
#         for Node in positions:
#             positions[Node][1] = UniqueSubgraphs[int(Node)-1].size()
#         nx.draw(ReducedPosetGraph, pos=positions, ax=axes, node_color="purple", edge_color="purple", font_color="white", with_labels=True)
#         plt.savefig(f"{GraphName}.Poset.svg")
#         plt.clf()
#         plt.close()
#         with open(f"{GraphName}.ReducedPosetGraph.pickle", "wb") as OutFile:
#             pickle.dump(ReducedPosetGraph, OutFile)
        
# #         Determine the unique list of largest subgraphs of both the red and blue subgraphs in a particular coloring
#         UniqueIntersectionSubgraphs = []
#         for Graph in CommonIntersectionSubgraphs:
#             Unique = True
#             for UniqueGraph in UniqueIntersectionSubgraphs:
#                 if nx.is_isomorphic(Graph, UniqueGraph):
#                     Unique = False
#                     break
#             if Unique:
#                 UniqueIntersectionSubgraphs.append(Graph)
        
# #         Save the list of unique subgraphs that are the largest subgraph in both red and blue subgraphs of each coloring
#         with open(f"{GraphName}.UniqueIntersectionSubgraphs.pickle", "wb") as OutFile:
#             pickle.dump(UniqueIntersectionSubgraphs, OutFile)
        
# #         Done with this K_n,m, return to the directory above
#         os.chdir(BaseDirectory)

In [24]:
# if os.path.exists("K_3,3"):
#     os.chdir("K_3,3")
#     with open("K_3,3.UniqueIntersectionSubgraphs.pickle","rb") as InputFile:
#         UniqueSubgraphs = pickle.load(InputFile)

#     _draw_graph_list(UniqueSubgraphs)
#     plt.close()
#     os.chdir(BaseDirectory)

In [25]:
# # Display all of the, up to isomorphism, largest graphs that are a subgraph of both red and blue subgraphs induced by a particular coloring of K_n,m
# for FileName in os.listdir(os.getcwd()):
#     if ".UniqueGraphs.pickle" in FileName:
# #         Get the n and m for the K_n,m host graph
#         n,m=FileName.split(".",1)[0].split("K_",1)[1].split(",")
#         n=int(n)
#         m=int(m)
#         HostGraph = nx.complete_multipartite_graph(n,m)
#         GraphName = f"K_{n},{m}"
#         print(f"Inspecting {GraphName}")
        
# #         Move into the correct directory
#         if os.path.exists(f"{GraphName}"):
#             os.chdir(f"{GraphName}")
#         else:
#             print("Skipping, since the graph folder has not been made yet")
#             continue
        
# #         Pick up the list of unique subgraphs that are the in the red and blue subgraphs of each coloring
#         if not os.path.exists(f"{GraphName}.UniqueIntersectionSubgraphs.pickle"):
#             print("Skipping, since the unique intersection subgraphs list has not been made yet")
#             continue
#         UniqueSubgraphs = []
#         CommonIntersectionSubgraphs = []
#         with open(f"{GraphName}.UniqueIntersectionSubgraphs.pickle", "rb") as InputFile:
#             UniqueIntersectionSubgraphs = pickle.load(InputFile)
        
#         _save_graph_list(UniqueIntersectionSubgraphs, f"{GraphName}.Unique.Intersections")
#         os.chdir(BaseDirectory)

In [26]:
print("Done")

Done
